<font size="6">Practical application</font>

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/cghiaus/ELECTRE_Tri/HEAD?labpath=ELECTRE_Tri%2FELECTRE_Tri_application.ipynb)

In [1]:
import numpy as np
import pandas as pd

import ELECTRE_Tri as et

# Set pandas to display 2 decimal places
pd.options.display.float_format = '{:.2f}'.format

# ELECTRE Tri: example with 3 alternatives and 2 criteria

The decision maker needs to provide:
- performance matrix;
- weights of criteria.

The parameters of ELECTRE Tri-B method (base profiles and thresholds) can be provided or deduced from the performance matrix.

## Performance matrix
### Alternatives
- A1: Basic renovation
- A2: Moderate renovation
- A3: Extensive renovation

### Criteria
- C1: Energy savings (kWh/m²/year): to be maximised
- C2: Investment cost (€/m²): to be minimised

### Values in performance matrix

| Alternative | C1/(kWh/m²/year) | C2/(€/m²) |
|-------------|-------------------|-----------|
| A1          | 50                | -100      |
| A2          | 80                | -200      |
| A3          | 120               | -350      |

**Note**: performance for criteria to be minimized are negative numbers.

In [2]:
data_file = './data/performance_matrix.csv'
print("Performance matrix")
A = pd.read_csv(data_file, index_col=0)
A

Performance matrix


,C1/(kWh/m²/year),C2/(€/m²)
Alternative,,
A1,50,-100
A2,80,-200
A3,120,-350


In [3]:
A.columns = ['C1', 'C2']

### Criteria weights

The weights of the criteria can be estimated by using Simos' method ([Figueira, Roy 2002](https://doi.org/10.1016/S0377-2217(01)00370-8)). See folder `\Simos_revised` of this repository. 

In [4]:
w = pd.Series({
    'C1': 0.7,
    'C2': 0.3
})
print('Weights \n')
w.to_frame().T.rename(index={0: 'w'})

Weights 



,C1,C2
w,0.70,0.30


## Calculate default parameters for ELECTRE Tri-B

### Statistics
- C1: μ = 83.33, σ = 35.12
- C2: μ = 216.67, σ = 125.83

In [5]:
A_stat = A.agg(['mean', 'std'])
A_stat

,C1,C2
mean,83.33,-216.67
std,35.12,125.83


### Categories and base profiles

We'll define 5 categories using μ and σ:

1. Very Poor: < μ - σ
2. Poor: μ - σ to μ - σ/2
3. Average: μ - σ/2 to μ + σ/2
4. Good: μ + σ/2 to μ + σ
5. Very Good: > μ + σ

using 4 base profiles (boundaries) between categories:
1. Very Poor: μ - σ
2. Poor: μ - σ/2
3. Good: μ + σ/2
4. Very Good: μ + σ

In [6]:
print("Base profiles")
# Define scaling factors for categories
scales = np.array([-1, -0.5, 0.5, 1])  # Very Poor, Poor, Good, Very Good

# Create DataFrame B with index
B = pd.DataFrame(index=['Very Poor', 'Poor', 'Good', 'Very Good'],
                 columns=A_stat.columns)

# Calculate values for each column using broadcasting
for col in A_stat.columns:
    mean_value = A_stat.loc['mean', col]
    std_value = A_stat.loc['std', col]
    B[col] = mean_value + scales * std_value

B

Base profiles


,C1,C2
Very Poor,48.21,-342.50
Poor,65.77,-279.58
Good,100.89,-153.75
Very Good,118.45,-90.84


### Thresholds
We'll calculate thresholds based on the interval between two base profiles:

For example, for C1, the interval between profiles is:
(118.45 - 48.21) / 4 = 17.56

The thresholds are:
- q (indifference): 10% of 17.56 = 1.76
- p (preference): 25% of 17.56 = 4.39
- v (veto): 50% of 17.56 = 8.78


In [7]:
# Calculate thresholds for each column
thresholds = {}
for col in B.columns:
    interval = (B[col].max() - B[col].min()) / B.shape[0]
    thresholds[col] = {
        'q': 0.10 * interval,
        'p': 0.25 * interval,
        'v': 0.50 * interval
    }

# Convert the thresholds dictionary to a DataFrame for clarity
T = pd.DataFrame(thresholds)

T

,C1,C2
q,1.76,6.29
p,4.39,15.73
v,8.78,31.46


In [8]:
opti, pessi = et.ELECTRE_Tri(A, B, T, w,
                              credibility_threshold=0.7)
print("\nOptimistic ranking:")
print(opti)

print("\nPessimistic ranking:")
print(pessi)


Optimistic ranking:
Alternative         A1   A2   A3
Very Poor >        NaN  NaN  NaN
(Very Poor, Poor)  NaN  NaN  NaN
(Poor, Good)       NaN 1.00  NaN
(Good, Very Good) 1.00  NaN 1.00
Very Good <        NaN  NaN  NaN

Pessimistic ranking:
Alternative         A1   A2   A3
Very Poor >        NaN  NaN  NaN
(Very Poor, Poor) 1.00  NaN 1.00
(Poor, Good)       NaN 1.00  NaN
(Good, Very Good)  NaN  NaN  NaN
Very Good <        NaN  NaN  NaN


In [9]:
print('Optimistic sorting')
et.sort(opti).to_frame(name="alternatives").rename_axis("categories")

Optimistic sorting


,alternatives
categories,
Very Poor >,[]
"(Very Poor, Poor)",[]
"(Poor, Good)",[A2]
"(Good, Very Good)","[A1, A3]"
Very Good <,[]


In [10]:
print('Pessimistic sorting')
et.sort(pessi).to_frame(name="alternatives").rename_axis("categories")

Pessimistic sorting


,alternatives
categories,
Very Poor >,[]
"(Very Poor, Poor)","[A1, A3]"
"(Poor, Good)",[A2]
"(Good, Very Good)",[]
Very Good <,[]


# Guidelines for chosing the parameters of ELECTRE Tri-B

## Categories

1. **Five-category system**:  
   This is a common and intuitive approach, often using categories like:
   - Very Bad
   - Bad
   - Neutral
   - Good
   - Very Good

   This structure is easy to understand and can be applied to many decision contexts.

2. **Seven-category system**:  
   This provides more granularity and can be structured as:
   - Extremely Poor
   - Very Poor
   - Poor
   - Average
   - Good
   - Very Good
   - Excellent

## Base profiles (categories boundaries)

For defining the boundaries or base profiles between these categories, some simple methods include:

1. **Equal interval method**:  
   Divide the range (maximum - minimum) of criterion values into equal intervals. For a five-category system, this would create boundaries at 20%, 40%, 60%, and 80% of the range.

2. **Percentile method**:  
   Use percentiles of the data distribution to set boundaries. For a five-category system, you might use the 20th, 40th, 60th, and 80th percentiles.

3. **Standard deviation method**:  
   Set boundaries based on standard deviations from the mean. For example, in a five-category system:
   - Very Bad: < μ - σ
   - Bad: μ - σ to μ - σ/2
   - Neutral: μ - σ/2 to μ + σ/2
   - Good: μ + σ/2 to μ + σ
   - Very Good: > μ + σ

   Where μ is the mean and σ is the standard deviation.

4. **Rounded natural breaks**:  
   Identify natural clusters in the data and round the boundaries to simple, easy-to-communicate values.

These simplified approaches can be particularly useful when:
- There's a lack of expert knowledge to define precise boundaries.
- The decision context doesn't require highly specific category definitions.
- You need a quick, initial categorization that can be refined later.
- Stakeholders prefer easily understandable and communicable category structures.

However, it's important to note that while these simplified approaches can be useful starting points, they may not always capture the nuances of complex decision problems. In many cases, they should be viewed as initial frameworks that can be adjusted based on expert input or more sophisticated analysis of the specific decision context.

## Thresholds

When setting indifference (q), preference (p), and veto (v) thresholds, simple percentage-based methods can provide a good starting point. These methods use the interval between two successive base profiles as a reference.

### Threshold ranges

1. **Indifference threshold (q)**:
   - Range: 5% to 15% of the interval between two successive base profiles
   - Typical value: 10% of the interval

2. **Preference threshold (p)**:
   - Range: 20% to 30% of the interval between two successive base profiles
   - Typical value: 25% of the interval

3. **Veto threshold (v)**:
   - Range: 40% to 60% of the interval between two successive base profiles
   - Typical value: 50% of the interval

Example calculation

Let's consider two successive base profiles for a criterion:
- Lower base profile: 100
- Upper base profile: 200
- Interval between profiles: 200 - 100 = 100

Threshold calculations using typical values:
- q = 10% of 100 = 10
- p = 25% of 100 = 25
- v = 50% of 100 = 50

### Guidelines for application

1. **Consistency**: Ensure that q < p < v for each criterion.

2. **Criterion-specific adjustments**: While these percentages provide a starting point, adjust them based on the nature of each criterion. For instance:
   - Criteria with high precision might use lower percentages.
   - Criteria with more uncertainty might use higher percentages.

3. **Rounding**: Round the calculated thresholds to a level of precision appropriate for the criterion.

4. **Stakeholder validation**: Present these calculated thresholds to stakeholders or experts for validation and refinement.

5. **Sensitivity analysis**: Test how small changes in these thresholds affect the final results to ensure robustness.

### Advantages and limitations

Advantages:
- Simplicity and ease of understanding
- Consistency across criteria
- Quick initial setup for further refinement
- Easily communicable to stakeholders

Limitations:
- May not capture the full complexity of some decision problems
- Might need adjustment for criteria with non-linear scales or special considerations

This method provides a structured starting point for setting thresholds in ELECTRE Tri-B, which can then be fine-tuned based on expert knowledge, data analysis, or specific requirements of the decision context.

# References

1. Daniel, S., & Ghiaus, C. (2023). Multi-Criteria Decision Analysis for Energy Retrofit of Residential Buildings: Methodology and Feedback from Real Application. Energies, 16(2), 902. https://doi.org/10.3390/en16020902 
2. The, A. N., & Mousseau, V. (2002). Using assignment examples to infer category limits for the ELECTRE TRI method. Journal of Multi‐Criteria Decision Analysis, 11(1), 29-43. DOI: https://doi.org/10.1002/mcda.314
3. Bouyssou, D., & Marchant, T. (2015). On the relations between ELECTRE TRI-B and ELECTRE TRI-C and on a new variant of ELECTRE TRI-B. European Journal of Operational Research, 242(1), 201-211. https://doi.org/10.1016/j.ejor.2014.09.057
4. Figueira, J., Roy, B. (2002). Determining the weights of criteria in the ELECTRE type methods with a revised Simos' procedure. European journal of operational research, 139(2), 317-326. https://doi.org/10.1016/S0377-2217(01)00370-8